# 欢迎来到NFL大数据碗的EDA现场

In [ ]:
import numpy as np
import pandas as pd

import math
import scipy
from scipy import stats
from scipy.spatial.distance import euclidean
from scipy.special import expit

from tqdm import tqdm

import random
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df_train = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', parse_dates=['TimeHandoff','TimeSnap'], infer_datetime_format=True, low_memory=False)

In [ ]:
df_train.columns

## 主观分析

In [ ]:
meta_df = pd.DataFrame({})

meta_df = meta_df.append([['GameId','比赛ID','分类','比赛','无','','测试与训练中的GameId都是不同的']])
meta_df = meta_df.append([['PlayId','回合ID','分类','比赛','无','','每个待预测数据都是一个唯一的回合Id']])
meta_df = meta_df.append([['Team','球队','分类','球队','中','','不同的球队有不同的进攻防守能力']])
meta_df = meta_df.append([['X','球员位置x','数值','球员动态','高','','球员位置决定了战术执行顺利与否']])
meta_df = meta_df.append([['Y','球员位置y','数值','球员动态','高','','球员位置决定了战术执行顺利与否']])
meta_df = meta_df.append([['S','球员速度','数值','球员动态','高','','最直接的说rusher的速度与码数的关系是很直接的']])
meta_df = meta_df.append([['A','球员加速度','数值','球员动态','高','','最直接的说rusher的加速度与码数的关系是很直接的']])
meta_df = meta_df.append([['Dis','','数值','球员动态','中','','']])
meta_df = meta_df.append([['Orientation','球员面向角度','数值','球员动态','低','','表现球员的观察方向，或者在更高级的维度会更有用']])
meta_df = meta_df.append([['Dir','球员移动角度','数值','球员动态','中','','移动方向感觉直接的作用不如间接的大']])
meta_df = meta_df.append([['NflId','球员Id','分类','球员静态','中','','根据具体球员能力不同，最终达成的码数不同']])
meta_df = meta_df.append([['DisplayName','球衣名字','分类','球员静态','无','','基本没用，更多是在可视化部分起作用']])
meta_df = meta_df.append([['JerseyNumber','球员号码','分类','球员静态','无','','一般决定了位置，但是位置有单独的字段，所以也没啥用']])
meta_df = meta_df.append([['Season','赛季','分类','比赛','无','','赛季看起来范围太大，应该没什么用']])
meta_df = meta_df.append([['YardLine','码线','分类','比赛','低','','看过比赛后我觉得码线有影响但是不大，是不是在终场前且分差很接近时会有呢']])
meta_df = meta_df.append([['Quarter','第几节','分类','比赛','低','','不认为第几节会有太大关系']])
meta_df = meta_df.append([['GameClock','比赛时间','时间','比赛','低','','同样不认为时间关系会很大']])
meta_df = meta_df.append([['PossessionTeam','进攻方','分类','比赛','中','','球队进攻防守能力有关']])
meta_df = meta_df.append([['Down','1~4 down','分类','比赛','中','','影响战术，通常如果是1选择保守，2,3会进攻性强一些，4则弃踢多']])
meta_df = meta_df.append([['Distance','需要多少码可以继续进攻','数值','比赛','中','','与Donw的关系类似']])
meta_df = meta_df.append([['FieldPosition','目前比赛在哪个队半场进行','分类','比赛','低','','这个信息在码线上也有体现']])
meta_df = meta_df.append([['HomeScoreBeforePlay','主队赛前积分','数值','球队','低','','这主要是体现球队实力']])
meta_df = meta_df.append([['VisitorScoreBeforePlay','客队赛前积分','数值','球队','低','','这主要是体现球队实力，影响应该是总体的']])
meta_df = meta_df.append([['NflIdRusher','持球人Id','分类','比赛','中','','持球人的影响肯定是单个特征中最大的']])
meta_df = meta_df.append([['OffenseFormation','进攻队形','分类','比赛','中','','不同的进攻方式通常目的是不同的，对应的码数也不同']])
meta_df = meta_df.append([['OffensePersonnel','进攻人员组成','分类','比赛','中','','这个主要是配合队形使用，可以认为是队形的更细分']])
meta_df = meta_df.append([['DefendersInTheBox','防守方在混战区的人数','数值','比赛','高','','这个人数跟战术有关，感觉有关系，其他kernel看关系还挺大']])
meta_df = meta_df.append([['DefensePersonnel','防守人员组成','分类','比赛','中','','防守人员是针对进攻人员来设置的']])
meta_df = meta_df.append([['PlayDirection','回合进行的方向','分类','比赛','无','','比赛方向，左还是右，关系不大']])
meta_df = meta_df.append([['TimeHandoff','传球时间','时间','比赛','低','','可能跟战术有关，或者进展是否顺序，一般来说越快越好']])
meta_df = meta_df.append([['TimeSnap','发球时间','时间','比赛','无','','感觉不到有什么关系，与handoff求时间差吧']])
meta_df = meta_df.append([['PlayerHeight','球员身高','数值','球员静态','低','','太明显感觉不到，但是不能说没有']])
meta_df = meta_df.append([['PlayerWeight','球员体重','数值','球员静态','低','','太明显感觉不到，但是不能说没有*2']])
meta_df = meta_df.append([['PlayerBirthDate','球员生日','时间','球员静态','无','','直接用没用，但是可以转为年龄，但是关系应该也不太大']])
meta_df = meta_df.append([['PlayerCollegeName','球员大学','分类','球员静态','低','','关系不大，虽然说名校可能实力更大，但是不尽然']])
meta_df = meta_df.append([['Position','球员职责','分类','球员静态','低','','根据持球人的Position或者有不错的效果']])
meta_df = meta_df.append([['HomeTeamAbbr','主队名','分类','球队','低','','聚合统计球队进攻防守能力']])
meta_df = meta_df.append([['VisitorTeamAbbr','客队名','分类','球队','低','','聚合统计球队进攻防守能力']])
meta_df = meta_df.append([['Week','第几周','分类','比赛','无','','目前是第几周，或者会考虑疲劳，但是缩小到每个回合，关系不大']])
meta_df = meta_df.append([['Stadium','球场','分类','环境','无','','微乎其微']])
meta_df = meta_df.append([['Location','球场所在位置','分类','环境','低','','可能有气候问题，比如NBA的掘金所在的高原地区']])
meta_df = meta_df.append([['StadiumType','球场类型','分类','环境','无','','微乎其微']])
meta_df = meta_df.append([['Turf','草皮','分类','环境','无','','微乎其微']])
meta_df = meta_df.append([['GameWeather','比赛天气','分类','环境','无','','微乎其微']])
meta_df = meta_df.append([['Temperature','温度','数值','环境','无','','微乎其微']])
meta_df = meta_df.append([['Humidity','湿度','数值','环境','无','','微乎其微']])
meta_df = meta_df.append([['WindSpeed','风速','数值','环境','无','','微乎其微']])
meta_df = meta_df.append([['WindDirection','风向','数值','环境','无','','微乎其微']])
meta_df = meta_df.append([['Yards','所获得的码数','数值','比赛','目标','','该次回合进攻方获得的码数，理论上多为整数，少数为负数或零']])

meta_df.columns = ['name','desc','type','segment','expectation','conclusion','comment']
meta_df.sort_values(by='expectation')

直观分析的Top10如下：
1. XYAS 四个动态特征；
2. DefendersInTheBox 表现防守方在Box里的人数；
3. PossessionTeam 进攻球队；
4. NflIdRusher 持球人Id；
5. Down 进攻次数；
6. OffenseFormation 进攻队形；
7. NflId 场上球员组成（比如对于NE，汤姆布雷迪伤了没上的影响应该是比较大的，等等）；

大体感觉是：球员动态 > 比赛 > 球队；

## 从Yards开始

In [ ]:
df_train.Yards.describe()

对`Yards`进行describe看整体情况，最小值有负数，这是正常的，发球后通常持球人都是在码线后，如何此时没能前进，而是被拦截，那么这一回合的码数就是负数，最大是99，几乎是跑完了全场，平均每回合推进4.2码，以这个码数看，通常进攻方要维持继续进攻是比较容易的，数据整体分布非正态，应该是右偏的；

In [ ]:
sns.distplot(df_train.Yards);

直方图上看还好，但是数据有不少为0的，这里做log转换要注意，应该是有微微右偏，且显示峰度；

In [ ]:
print("Skewness: %f" % df_train.Yards.skew())
print("Kurtosis: %f" % df_train.Yards.kurt())

确实是正偏+显示峰度，与上述可视化结果一致；

## 再来看看Top10与Yards的关系

### 数值型

In [ ]:
df_train_rusher = df_train[df_train.NflId==df_train.NflIdRusher].copy()

df_train_rusher['ToLeft'] = df_train_rusher.PlayDirection.apply(lambda play_direction:play_direction=='left')
df_train_rusher['X_std'] = df_train_rusher[['ToLeft','X']].apply(lambda row:120-row.X-10 if row.ToLeft else row.X-10, axis=1)
df_train_rusher['Y_std'] = df_train_rusher[['ToLeft','Y']].apply(lambda row:160/3-row.Y if row.ToLeft else row.Y, axis=1)

In [ ]:
var = 'X_std'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

In [ ]:
meta_df.loc[meta_df.name=='X','conclusion'] = '可以生成一个规则特征，用于对最终结果的限制，另外作为空间位置信息有待挖掘'

对于`X`的处理需要标准化后，否则结果是相对的而不是绝对的，通过对`X_std`的散点图看到，**能够得到超大码数的前提是你距离TouchDown区域够远**，否则是不可能得到大码数的，这一特征需要构建出来，因为这是规则性的东西，可以加到结果限制中；

In [ ]:
var = 'Y_std'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

In [ ]:
meta_df.loc[meta_df.name=='Y','conclusion'] = '可以生成一个距离左右边界距离的特征，作为空间位置有待更深入的挖掘'

对`Y_std`的可视化看到，大码数集中在中间区域，而两边则只有相对小的码数，不过因为数据主要集中在中间区域，所以这一特点并不明显，有限的数据或许可以说明以下观点：**当持球人在Y方向的中间时，他左右可以选择的空间更大，因此他更有希望通过左右跑动摆脱获取更大的码数**，因此可以考虑生成一个距离左右边界的距离特征来更清晰的表达这一点，且与目标成线性关系，目前是非线性的；

In [ ]:
var = 'S'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

In [ ]:
meta_df.loc[meta_df.name=='S','conclusion'] = '没有明显线性关系，大码数集中在中间部分，所以速度适中是个选项'

没看到想象中的线性关系，说明该问题实际上是比较复杂的问题，单个特征的线性关系不容易获取，主要也是因为`S`是短时间内测量的，并不是说每个球员就一直处于这个速度，因此参考意义没有字面意义那么大，可以看出的是：**相对来说速度适中的情况下，码数获取更大，可以认为是速度太慢容易被拦截，速度太快不同意改变自己的方向，也就是缺少变化，而速度适中时，既能向前冲刺，又可以做适当的方向上的调整**；

In [ ]:
var = 'A'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

In [ ]:
meta_df.loc[meta_df.name=='A','conclusion'] = '没有明显线性关系，大码数集中在左边，也就是说球员速度更平均时'

加速度可视化呈现出一种诡异的拐角型，两头小，中间大，这里与速度类似的是，加速度太大，反而没能获取太大的码数，原因我认为是类似的，可能需要考虑结合二者生成一个能表达这部分信息的特征；

数值型的小结：没有明显的线性关系存在，说明如果特征直接送模型，效果不会理想，需要深度挖掘、组合高维特征、更具业务意义的操作，但是也从4个主要的动态数值信息中获取到一些有趣的信息；

PS：注意XYSA都只可视化了持球人的部分，这是因为我认为对于持球人、进攻方其他球员、防守方球员来说，毫无疑问最重要的是持球人部分，但是这三部分的含义相差很大，放到一起可视化可能无法得到有用的信息，因此这里只对持球人部分做可视化；

### 分类型

In [ ]:
var = 'DefendersInTheBox'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

In [ ]:
meta_df.loc[meta_df.name=='DefendersInTheBox','conclusion'] = '不看超大码数时，人数越多，码数越小'

能看到以下的数据信息：
- 超大码数来自于防守方在Box中人数为5到9人时；
- 同样在5到9人时，也更多的出现负码数，也就是迫使进攻方失误；
- 大码数在这个特征维度上，属于异常值，说明本身应该是很少出现的；
- 该特征在与码数的关系上，整体有一个人数越多，码数越小的趋势（排除异常值的情况下）；

In [ ]:
var = 'PossessionTeam'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(20, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

In [ ]:
meta_df.loc[meta_df.name=='PossessionTeam','conclusion'] = '球队层面对码数的影响不大，各只球队基本持平'

进攻球队的维度上看，基本差异不明显，说明整体战术是一致的，争取小码数推进，有机会就拿大码数，但是不冒险，因此仅仅区分球队无法看到更多信息；

In [ ]:
var = 'NflIdRusher'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(30, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

In [ ]:
meta_df.loc[meta_df.name=='NflIdRusher','conclusion'] = '球员层面区别很大，但是相对来说球员层面的数据量更小，所以是否具有代表性有待研究'

各个球员之间的差异明显，远远超过球队之间的差异，因此NflIdRusher应该是一个比较重要的特征，这证明了主观分析部分对这个特征的判断；

In [ ]:
var = 'Down'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

In [ ]:
meta_df.loc[meta_df.name=='Down','conclusion'] = 'Down越大，代表所剩机会越少，一般码数都会越小'

对于`Down`来说，有一点明显的特点在于第一次进攻大码数肯定比第二次多，以此类推至第四次，这是因为在第四次如果没有弃踢，那么通常也是因为目前所需的码数很小，所以一般选择最保守的战术获取小码数即可，没必要高风险追求大码数；

In [ ]:
var = 'OffenseFormation'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

In [ ]:
meta_df.loc[meta_df.name=='OffenseFormation','conclusion'] = '数据分布主体是类似的，大码数方向根据队形不同有所差异'

对于`OffenseFormation`，每种进攻队形的主体都是获取小码数，这也说明了在NFL这种高水平对抗中，大码数出现是很困难，以至于不算为主要进攻手段；

分类型的小结：依然没能看到与码数特别相关的特征，这也说明了问题的复杂程度不是单个变量能直接相关的，事实上，很多变量与码数的相关性都是接近于0的，所以我们的目的应该是构建一个NFL的`OverallQual`特征，如果你做过HousePrice项目，你知道我说的是什么；

最后看一下对特征信息的更新：

In [ ]:
meta_df

## 客观分析所有变量

同样的，在这一步，只使用rusher部分，由于这里的数据是展开了的，最终结果是一个回合为一行的，而除了球员信息外，其他部分都是一致的，而球员信息与结果关系最大的理应是rusher的动态静态信息；

In [ ]:
#correlation matrix
corrmat = df_train_rusher.corr()
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corrmat, vmax=.8, square=True);

热图中看到的信息如下：
- 没有与`Yards`特别相关的，不管正负；
- `Distance`与`Down`负相关；
- `Week`与`Temperature`负相关；
- `Season`、`GameId`、`PlayId`是相关的；
-  环境类更是无限接近于0；

In [ ]:
k = 10
cols = corrmat.nlargest(k, 'Yards')['Yards'].index
cm = np.corrcoef(df_train_rusher[cols].values.T)
plt.subplots(figsize=(8, 8))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

看到以下信息：
- 最相关的是`A`；
- `S`与`Dis`相关性高达0.88，去掉`Dis`；
- `GameId`与`PlayId`、`Season`相关性都很高，由于`GameId`与`PlayId`都是不会重复出现在测试集的，因此保留`Season`；
- `NflId`在这里表示的应该是`NflIdRusher`；

注意因为此处是相关性分析，对于分类型特征可能不是很**友好**；

最终这一步后保留的特征是：`A`、`S`、`Distance`、`YardLine`、`Season`、`NflIdRusher`；

In [ ]:
sns.set()
cols = ['Yards', 'A', 'S', 'Distance', 'YardLine', 'Season', 'NflIdRusher']
sns.pairplot(df_train_rusher[cols], size = 2.5)
plt.show();

目前看这个可视化图，如果一些跟规则相关的分割线外，看不到太多有用的信息，这一点从相关系数上也能看出，最高才0.1+，说明很多特征都是很基础、原始的，哟要体现出较强的关联性，一个靠EDA的业务探索，一个靠FE的工程化特征构建；

## 缺失数据处理

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data.Total>0]

分析下缺失情况：
- 最多的是WindDirection和WindSpeed，属于环境特征，缺失大于13%，由于本身相关性不高，且缺失较多，因此环境类的做drop处理；
- Temperature、GameWeather、Humidity同上；
- StadiumType理想状态是通过google填充，但是由于其低相关性，且可能携带噪声，同样drop处理；
- FieldPosition的缺失是因为在中线开球，所以填充其middle即可；
- OffenseFormation进攻队形；
- Dir移动角度影响球员后续的位置；
- Orientation面向角度说实话不知道怎么用这个字段，但是缺失不多，drop太可惜；
- DefendersInTheBox在Box的人数算是比较重要的，这个可以通过计算得到；

后5个需要填充，FieldPosition简单，后4个需要考虑，但是其实缺失都很少，所以影响也不会太大，但是不建议删除行，因为22行为一个回合组，删除掉可能会影响后续的聚合分析；

In [ ]:
df_train = df_train.drop(['WindSpeed','WindDirection','Temperature','GameWeather','Humidity','StadiumType'], axis=1)

In [ ]:
df_train.FieldPosition = df_train.FieldPosition.fillna('middle')

In [ ]:
# OffenseFormation, Dir, Orientation, DefendersInTheBox
# df_train[]

In [ ]:
df_train.isnull().sum().max()

## 异常数据处理

主要关注一些离群点与表现明显不正常的点，注意这里单变量分析主要是分析目标变量，但是这里的目标变量由于有球场做限制，因此不存在所谓的异常值，而二元分析应该是偏离主分布的点，但是目前看二元的分布没有明显的线性等关系，因此这一步先省略；

In [ ]:
from sklearn.preprocessing import StandardScaler
#standardizing data
saleprice_scaled = StandardScaler().fit_transform(df_train['Yards'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

看到虽然有14这种明显大于0的点，但是由于确实存在这种可能（几乎跑了全场的情况），因此依然是正常数据而非异常数据，当然了，属于小概率情况，实际上如何去掉**超大码数**和**负码数**也许会更好；

## 数据测试假设检查

主要是正态性检查、同质性检查、线性检查（这个比较麻烦，目前数据没有明显线性关系）、不存在相关误差；

### 从正态性开始

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train['Yards'], fit=stats.norm);
fig = plt.figure()
res = stats.probplot(df_train['Yards'], plot=plt)

能够看到码数分布显示峰度，且有正偏的，且概率图上看也与红线相差很远，但是无法直接应用log变化，因为数据中有不少在0和0以下的情况没法处理；

## 深度EDA

从此开始挖掘跟橄榄球相关的更深层次的特征，并通过可视化等手段验证其有效性；

In [ ]:
df_train_rusher = df_train[df_train.NflId==df_train.NflIdRusher].copy()

### 先来看看rusher的位置热图

In [ ]:
plt.subplots(figsize=(16, 8))
tmp = df_train_rusher[["Y", "X", "Yards"]].copy()
tmp.X = pd.cut(tmp.X, 12*5)
tmp.Y = pd.cut(tmp.Y, 6*5)
sns.heatmap(tmp.groupby(['Y','X']).count().reset_index().pivot("Y", "X", "Yards"))

通常来说美式橄榄球有两种进攻方式：
- 通过四分卫持球，前传找合法接球人，后由该接球人持球推进；
- 通过四分卫将球后传给跑卫，由跑卫持球冲阵；

目前这个问题下对应的数据都是第二种情况，对于第二种进攻方式，一般来说失误更少，但是获取码数相对较小，容易被拦截但是也不容易丢球，比较保守的战术，从热图中看，`Y`从22到30是选择比较多的区域，这个区域相对横向的可移动范围比较大，同时又微微**避开了中间的混战区**，看起来像是个不错的选择；

### 再来看看各位置rusher对应拿到的码数

In [ ]:
plt.subplots(figsize=(16, 8))
sns.heatmap(tmp.groupby(['Y','X']).mean().reset_index().pivot("Y", "X", "Yards"), center=0, vmin=-5, vmax=10)

注意，由于是求各个位置的平均值，因此不乏有些偏远位置数据量很小，球队出的奇招怪招啥的，这部分要特别注意，主要还是分析中间这一块大区域的情况：
- 数据相对比较稳定，各区域差异不大；
- 主体的边缘上颜色变化较大，包括负数和大数；
- 主体以外要么是大码数，要么是负码数，这部分就比较凌乱，说明特别的战术通常都是高风险与高收益并存；

### 开始挖掘 - 热热身

下面是之前的主观分析后产生了结论信息的特征；

In [ ]:
meta_df[meta_df.conclusion.str.len()>0]

#### 挖掘rusher的球员静态信息

- 身高
- 体重
- 年龄

In [ ]:
df_train_rusher.PlayerHeight = df_train_rusher.PlayerHeight.apply(lambda height:int(height[0])*12+int(height[2:])).astype('int')
df_train_rusher['Age'] = df_train_rusher.PlayerBirthDate.apply(lambda bd:2019-int(bd[-4:]))

In [ ]:
plt.subplots(figsize=(20, 5))

plt.subplot(1,3,1)
var = 'PlayerHeight'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
#data.plot.scatter(x=var, y='Yards')
#axs[0][0].plot.scatter(data[var],data['Yards'])
plt.scatter(data[var],data['Yards'])

plt.subplot(1,3,2)
var = 'PlayerWeight'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
plt.scatter(data[var],data['Yards'])

plt.subplot(1,3,3)
var = 'Age'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
plt.scatter(data[var],data['Yards'])

可以看到：
- 身高、体重中都体现出在中间部分的数据获取到的大码数较多；
- 年龄体现出年轻选手更容易获取大码数，峰值出现在24岁左右时，这一点从跑卫黄金年龄可以看出，实际上除了四分卫，NFL其他大部分位置都很难打到30岁以后，壮哉，**汤姆布雷迪**；

#### 挖掘距离左右边界特征`Y_dis`

In [ ]:
df_train_rusher['Y_dis'] = np.abs(np.abs(df_train_rusher.Y - df_train_rusher.Y.mean()) - df_train_rusher.Y.mean())

In [ ]:
var = 'Y_dis'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

与之前的码数热图基本对应，距离左右边界越远，或者说越接近球场纵向的中间区域，相对码数会比较稳定集中，且产生大码数；

#### 看看`Dir`和`Orientation`的夹角

In [ ]:
df_train_rusher['Dir_orientation'] = np.abs(df_train_rusher.Dir - df_train_rusher.Orientation)
df_train_rusher['Dir_orientation'] = df_train_rusher['Dir_orientation'].apply(lambda do:360-do if do > 180 else do)

In [ ]:
var = 'Dir_orientation'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

从球员移动方向与面向夹角来看，度数越大（注意这里夹角做了处理，避免出现大于180度的情况），相对获取码数越小，我理解如果rusher正在往前跑，却往其他方向看，多数情况下说明有防守球员正在向他冲来，而空间环境更好的rusher，通常是不需要左顾右盼；

#### 开球到传球的时间 - (`TimeHandoff`-`TimeSnap`)

In [ ]:
df_train_rusher['TimeFromSnapToHandoff'] = (df_train_rusher.TimeHandoff - df_train_rusher.TimeSnap).apply(
    lambda x:x.total_seconds()).astype('int8')

In [ ]:
var = 'TimeFromSnapToHandoff'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

从传球到开球的时间差来看，时间大于2秒时，通常结果都不会太好，但是由于大量数据集中在1和2秒上，而这部分差异不明显，因此提供的有效信息并不多；

#### 比赛进行时长、Quarter

In [ ]:
df_train_rusher['GameDuration'] = (df_train_rusher.GameClock.apply(
    lambda gc:15*60-int(gc[:2])*60-int(gc[3:5]))) + (df_train_rusher.Quarter-1)*15*60

In [ ]:
var = 'GameDuration'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

In [ ]:
var = 'Quarter'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

可以获取到如下信息：
- 常规时间内各节间码数分布基本一致；
- 常规时间与加时赛之间有3分钟休息，这就是上图空白部分的原因；
- 加时赛相对常规赛码数分布更加集中在小码数，从箱行图上看同样可以获取到这个结果；

NFL加时赛规则：与常规时间不同的是引入了突然死亡规则，即在加时赛过程中，任何一方率先达阵，则直接宣告胜利，因此对于进攻方来说，要严格控制失误，保持继续进攻，则优势会很大，因为犯错的成本是很高的，因此一般战术选择会相对常规时间更加保守；

#### 距离达阵还有多少码

In [ ]:
df_train_rusher['DistanceTouchDown'] = df_train_rusher[['YardLine','FieldPosition','PossessionTeam']].apply(
    lambda yfp:100-yfp['YardLine'] if(yfp['PossessionTeam']==yfp['FieldPosition']) else yfp['YardLine'], axis=1)

In [ ]:
var = 'DistanceTouchDown'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
data.plot.scatter(x=var, y='Yards');

这里的斜线可以看到是达阵线，即达到的码数等于达阵所需的码数，很明显的趋势时，所需码数越小，达阵线会越密集，这也是合理的，毕竟需要5码达阵和需要50码对于进攻方来说不是一个难度；

#### rusher与码线的横向距离

#### DL-LB防守阵型

In [ ]:
# Create the DL-LB combos
# Clean up and convert to DL-LB combo
df_train_rusher['DL_LB'] = df_train_rusher['DefensePersonnel'].str[:10].str.replace(' DL, ','-').str.replace(' LB','')
top_5_dl_lb_combos = df_train_rusher.groupby('DL_LB').count()['GameId'].sort_values().tail(10).index.tolist()

In [ ]:
var = 'DL_LB'
data = pd.concat([df_train_rusher.loc[df_train_rusher['DL_LB'].isin(top_5_dl_lb_combos)].Yards, 
                  df_train_rusher.loc[df_train_rusher['DL_LB'].isin(top_5_dl_lb_combos)][var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)
fig.set_ylim(-10,20)

从DL_LB组合分类来看码数：
- 各组之间存在一定的差异，注意这里y轴是zoom到-10和20的，所以这个差异看起来很明显，实际上大部分数据也都集中在这个区域；
- 3-2组合时，进攻方码数平均最高，而4-4则把对方的码数压制到最低；
- 而3-3,5-2,4-2,2-4情况基本一致；

#### rusher的Position

In [ ]:
var = 'Position'
data = pd.concat([df_train_rusher.Yards, df_train_rusher[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

看到对于rusher不同的Position来说，有以下信息：
- 各位置拿下的平均码数差异明显，当然要注意的是数据量的差异，RB肯定是最多的；
- RB作为最常冲阵的位置，码数处于中间位置，不高不低；
- CB码数最高；
- DT、DE、G数据少的可以忽略了都；

#### 距离rusher最近的队友多远、对手多远、队友对手距离多远

In [ ]:
possessionteam_map = {
    'BLT':'BAL',
    'CLV':'CLE',
    'ARZ':'ARI',
    'HST':'HOU'
}
df_train.PossessionTeam = df_train.PossessionTeam.apply(lambda pt:possessionteam_map[pt] if pt in possessionteam_map.keys() else pt)

In [ ]:
df_train['TeamBelongAbbr'] = df_train.apply(
    lambda row:row['HomeTeamAbbr'] if row['Team']=='home' else row['VisitorTeamAbbr'],axis=1)

df_train['Offense'] = df_train.apply(lambda row:row['PossessionTeam']==row['TeamBelongAbbr'],axis=1)

In [ ]:
df_aggregation = pd.DataFrame(columns={
    'GameId':[],'PlayId':[],'Teammate_dis':[],'Enemy_dis':[],'Teamate_enemy_dis':[],'Nearest_is_teammate':[],'Yards':[]})

for k,group in df_train.groupby(['GameId','PlayId']):
    rusher = group[group.NflId==group.NflIdRusher].iloc[0]
    offenses = group[group.NflId!=group.NflIdRusher][group.Offense]
    defenses = group[~group.Offense]
    def get_nearest(target, df):
        df['Tmp_dis'] = df[['X','Y']].apply(
            lambda xy:np.linalg.norm(np.array([xy.X,xy.Y])-np.array([rusher.X,rusher.Y])), 
            axis=1)
        return df.sort_values(by='Tmp_dis', ascending=False).iloc[0]
    nearest_offense = get_nearest(rusher, offenses)
    nearest_defense = get_nearest(rusher, defenses)
    Teamate_enemy_dis = np.linalg.norm(np.array([nearest_offense.X,nearest_offense.Y])-np.array([nearest_defense.X,nearest_defense.Y]))
    
    df_aggregation = df_aggregation.append(
        {'GameId':k[0],'PlayId':k[1],
         'Teammate_dis':nearest_offense.Tmp_dis,
         'Enemy_dis':nearest_defense.Tmp_dis,
         'Teamate_enemy_dis':Teamate_enemy_dis,
         'Nearest_is_teammate': 1 if nearest_offense.Tmp_dis < nearest_defense.Tmp_dis else 0,
        'Yards':rusher.Yards}, ignore_index=True)
    
df_aggregation.info()

In [ ]:
plt.subplots(figsize=(20, 5))

plt.subplot(1,3,1)
var = 'Teammate_dis'
data = pd.concat([df_aggregation.Yards, df_aggregation[var]], axis=1)
plt.scatter(data[var],data['Yards'])

plt.subplot(1,3,2)
var = 'Enemy_dis'
data = pd.concat([df_aggregation.Yards, df_aggregation[var]], axis=1)
plt.scatter(data[var],data['Yards'])

plt.subplot(1,3,3)
var = 'Teamate_enemy_dis'
data = pd.concat([df_aggregation.Yards, df_aggregation[var]], axis=1)
plt.scatter(data[var],data['Yards'])

可以看到以下信息：
- rusher的最近的队友离他距离在10到25码时，拿到的码数相对更大，队友过于接近他时，码数反而比较小；
- rusher的最近的对手离他距离在17到28码时，拿到的码数相对更大；
- 对于队友与对手的距离，没有体现出太多与码数有关的信息；

In [ ]:
var = 'Nearest_is_teammate'
data = pd.concat([df_aggregation.Yards, df_aggregation[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="Yards", data=data)

从最接近rusher的球员是队友还是对手上看：
- 为对手时：码数相对较小，大码数则远少于为对手时；
- 为队友时，码数相对较大，当然这个差异是很小的，但是大码数的情况则远远多于前者；

### 利用Voronoi、空间影响力热图分析空间信息 - show time

- 通过可视化球场动态，对比码数大小不同时的球员的区别；
- 利用VOronoi求rusher可控空间大小、是否与对手空间直接接触；
- 如何通过位置、速度、加速度、角度量化以rusher为中心的对整个球场的控制力影响图；

#### VIP HIT from [kernel](https://www.kaggle.com/pednt9/vip-hint-coded)

这个kernel是对官方kernel中的VIP Hit的代码实现，实现了基本的基于球的控制力计算、热图绘制；

如果这个控制力计算方式是正确的，那么如何去使用这个信息来构建特征呢，记录以下方式：
- 计算rusher位置的控制力，目前从1000个结果看效果不明显；
- 计算rusher方向路径上的控制力平均值 - todo

In [ ]:
def standardize_dataset(train):
    train['ToLeft'] = train.PlayDirection == "left"
    train['IsBallCarrier'] = train.NflId == train.NflIdRusher
    train['TeamOnOffense'] = "home"
    train.loc[train.PossessionTeam != train.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    train['IsOnOffense'] = train.Team == train.TeamOnOffense # Is player on offense?
    train['YardLine_std'] = 100 - train.YardLine
    train.loc[train.FieldPosition.fillna('') == train.PossessionTeam,  
            'YardLine_std'
             ] = train.loc[train.FieldPosition.fillna('') == train.PossessionTeam,  
              'YardLine']
    train['X_std'] = train.X
    train.loc[train.ToLeft, 'X_std'] = 120 - train.loc[train.ToLeft, 'X'] 
    train['Y_std'] = train.Y
    train.loc[train.ToLeft, 'Y_std'] = 53.3 - train.loc[train.ToLeft, 'Y'] 
    train['Orientation_std'] = train.Orientation
    train.loc[train.ToLeft, 'Orientation_std'] = np.mod(180 + train.loc[train.ToLeft, 'Orientation_std'], 360)
    train['Dir_std'] = train.Dir
    train.loc[train.ToLeft, 'Dir_std'] = np.mod(180 + train.loc[train.ToLeft, 'Dir_std'], 360)
    train.loc[train['Season'] == 2017, 'Orientation'] = np.mod(90 + train.loc[train['Season'] == 2017, 'Orientation'], 360)    
    
    return train

df_train2 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
dominance_df = standardize_dataset(df_train2)
dominance_df['Rusher'] = dominance_df['NflIdRusher'] == dominance_df['NflId']

In [ ]:
def radius_calc(dist_to_ball):
    ''' I know this function is a bit awkward but there is not the exact formula in the paper,
    so I try to find something polynomial resembling
    Please consider this function as a parameter rather than fixed
    I'm sure experts in NFL could find a way better curve for this'''
    return 4 + 6 * (dist_to_ball >= 15) + (dist_to_ball ** 3) / 560 * (dist_to_ball < 15)

class Controller:
    '''This class is a wrapper for the two functions written above'''
    def __init__(self, play):
        self.play = play
        self.vec_influence = np.vectorize(self.compute_influence)
        self.vec_control = np.vectorize(self.pitch_control) 
        
    def compute_influence(self, x_point, y_point, player_id):
        '''Compute the influence of a certain player over a coordinate (x, y) of the pitch
        '''
        point = np.array([x_point, y_point])
        player_row = self.play.loc[player_id]
        theta = math.radians(player_row[56])
        speed = player_row[5]
        player_coords = player_row[54:56].values
        ball_coords = self.play[self.play['IsBallCarrier']].iloc[:, 54:56].values

        dist_to_ball = euclidean(player_coords, ball_coords)

        S_ratio = (speed / 13) ** 2         # we set max_speed to 13 m/s
        RADIUS = radius_calc(dist_to_ball)  # updated

        S_matrix = np.matrix([[RADIUS * (1 + S_ratio), 0], [0, RADIUS * (1 - S_ratio)]])
        R_matrix = np.matrix([[np.cos(theta), - np.sin(theta)], [np.sin(theta), np.cos(theta)]])
        COV_matrix = np.dot(np.dot(np.dot(R_matrix, S_matrix), S_matrix), np.linalg.inv(R_matrix))

        norm_fact = (1 / 2 * np.pi) * (1 / np.sqrt(np.linalg.det(COV_matrix)))    
        mu_play = player_coords + speed * np.array([np.cos(theta), np.sin(theta)]) / 2

        intermed_scalar_player = np.dot(np.dot((player_coords - mu_play),
                                        np.linalg.inv(COV_matrix)),
                                 np.transpose((player_coords - mu_play)))
        player_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_player[0, 0])

        intermed_scalar_point = np.dot(np.dot((point - mu_play), 
                                        np.linalg.inv(COV_matrix)), 
                                 np.transpose((point - mu_play)))
        point_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_point[0, 0])

        return point_influence / player_influence
    
    
    def pitch_control(self, x_point, y_point):
        '''Compute the pitch control over a coordinate (x, y)'''

        offense_ids = self.play[self.play['IsOnOffense']].index
        offense_control = self.vec_influence(x_point, y_point, offense_ids)
        offense_score = np.sum(offense_control)

        defense_ids = self.play[~self.play['IsOnOffense']].index
        defense_control = self.vec_influence(x_point, y_point, defense_ids)
        defense_score = np.sum(defense_control)

        return expit(offense_score - defense_score)
    
    def display_control(self, grid_size=(50, 30), figsize=(12, 8)):
        front, behind = 30, 10
        left, right = 30, 30

        if self.play['IsOnOffense'].iloc[0]==True:
            colorm = ['purple'] * 11 + ['yellow'] * 11
        else:
            colorm = ['yellow'] * 11 + ['purple'] * 11
#         colorm = ['purple'] * 11 + ['yellow'] * 11
        colorm[np.where(self.play.Rusher.values)[0][0]] = 'black'
        player_coords = self.play[self.play['Rusher']][['X_std', 'Y_std']].values[0]

        X, Y = np.meshgrid(np.linspace(player_coords[0] - behind, 
                                       player_coords[0] + front, 
                                       grid_size[0]), 
                           np.linspace(player_coords[1] - left, 
                                       player_coords[1] + right, 
                                       grid_size[1]))

        # infl is an array of shape num_points with values in [0,1] accounting for the pitch control
        infl = self.vec_control(X, Y)

        plt.figure(figsize=figsize)
        plt.contourf(X, Y, infl, 12, cmap='bwr')
        plt.scatter(self.play['X_std'].values, self.play['Y_std'].values, c=colorm)
        plt.title('Yards gained = {}, play_id = {}'.format(self.play['Yards'].values[0], 
                                                           self.play['PlayId'].unique()[0]))
        plt.show()

In [ ]:
_play_id1 = random.choice(dominance_df[~dominance_df.ToLeft].PlayId.tolist())
my_play = dominance_df[dominance_df.PlayId==_play_id1].copy()
control = Controller(my_play)
coords = my_play.iloc[1, 54:56].values         # let's compute the influence at the location of the first player
_pitch_control = control.vec_control(*coords)
print(_pitch_control)
control.display_control()

In [ ]:
_play_id2 = random.choice(dominance_df[~dominance_df.ToLeft].PlayId.tolist())
my_play2 = dominance_df[dominance_df.PlayId==_play_id2].copy()
control2 = Controller(my_play2)
control2.display_control()

In [ ]:
_controls = []
_yards = []
for _play_id in dominance_df.PlayId.unique().tolist()[:10000]:
    _my_play = dominance_df[dominance_df.PlayId==_play_id].copy()
    _control = Controller(_my_play)
    _rusher = _my_play.query('Rusher == True').iloc[0]
    coords = (_rusher.X_std,_rusher.Y_std)
    _pitch_control = _control.vec_control(*coords)
    _controls.append(_pitch_control)
    _yards.append(_rusher.Yards)
    
plt.scatter(_controls, _yards)

#### 其他kernel借鉴

- [DL_LB](https://www.kaggle.com/robikscube/big-data-bowl-comprehensive-eda-with-pandas)
- [更真实的球场可视化](https://www.kaggle.com/robikscube/nfl-big-data-bowl-plotting-player-position)
- [持球人的Position](https://www.kaggle.com/jaseziv83/comprehensive-cleaning-and-eda-of-all-variables)
- [VIP Hit](https://www.kaggle.com/pednt9/vip-hint-coded)

## The end.